## **finetuning trocr and crnn , ocr model tensorfelow 2.15 version**

In this notebook, we initially worked with our OCR model using TensorFlow 2.15, along with fine-tuning CRNN and TrOCR. However, we estimated that fine-tuning these models would take over 30 hours and  the session crashed. Given this time constraint, we decided to proceed with YOLO for character recognition instead

In [ ]:
import os
import shutil
import xml.etree.ElementTree as ET
import cv2
import matplotlib.pyplot as plt
import random
import numpy as np
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import glob
from PIL import Image
#from ultralytics import YOLO
from google.colab.patches import cv2_imshow
# import easyocr
# import pytesseract
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/plate_image_with_dummy-train.zip' -d plate_train

Streaming output truncated to the last 5000 lines.
  inflating: plate_train/train/dummy_te_14501.jpg  
  inflating: plate_train/train/dummy_te_14501.xml  
  inflating: plate_train/train/dummy_te_14502.jpg  
  inflating: plate_train/train/dummy_te_14502.xml  
  inflating: plate_train/train/dummy_te_14503.jpg  
  inflating: plate_train/train/dummy_te_14503.xml  
  inflating: plate_train/train/dummy_te_14504.jpg  
  inflating: plate_train/train/dummy_te_14504.xml  
  inflating: plate_train/train/dummy_te_14505.jpg  
  inflating: plate_train/train/dummy_te_14505.xml  
  inflating: plate_train/train/dummy_te_14506.jpg  
  inflating: plate_train/train/dummy_te_14506.xml  
  inflating: plate_train/train/dummy_te_14507.jpg  
  inflating: plate_train/train/dummy_te_14507.xml  
  inflating: plate_train/train/dummy_te_14508.jpg  
  inflating: plate_train/train/dummy_te_14508.xml  
  inflating: plate_train/train/dummy_te_14509.jpg  
  inflating: plate_train/train/dummy_te_14509.xml  
  inflating: 

In [ ]:
def folder_organizer(folder_path):
    # Define target directory
    images_dir = os.path.join(folder_path, "images")
    labels_dir = os.path.join(folder_path, "labels")

    # Create folders if don't exist
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)

    for file_name in os.listdir(folder_path):
        if file_name in ("images", "labels"):
            continue

        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            lower_name = file_name.lower()
            if lower_name.endswith(('.jpg')):
                try:
                    shutil.move(file_path, os.path.join(images_dir, file_name))
                except Exception as e:
                    print(f"Image transfer error {file_name}: {e}")
            elif lower_name.endswith('.xml'):
                try:
                    shutil.move(file_path, os.path.join(labels_dir, file_name))
                except Exception as e:
                    print(f"XML transfer error {file_name}: {e}")

    print(f'✅ The organization successfully done at "{folder_path}".')


folder_organizer('/content/plate_train/train')

✅ The organization successfully done at "/content/plate_train/train".


In [ ]:
def count_files(folder):
    print(f"📂 {folder}:")
    print(" images:", len(os.listdir(os.path.join(folder, "images"))))
    print(" labels:", len(os.listdir(os.path.join(folder, "labels"))))

count_files('/content/plate_train/train')

📂 /content/plate_train/train:
 images: 45469
 labels: 45469


In [ ]:
classes = {'کل ناحیه پلاک': 0}

def update_classes(folder_path):
    global classes
    labels_dir = os.path.join(folder_path, "labels")

    for xml_file in os.listdir(labels_dir):
        xml_path = os.path.join(labels_dir, xml_file)
        try:
            tree = ET.parse(xml_path)
            root = tree.getroot()

            for obj in root.findall('.//object'):
                name = obj.find('name').text
                if name not in classes:
                    new_id = len(classes)
                    classes[name] = new_id
        except ET.ParseError as e:
            print(f"Error parsing {xml_file}: {e}")
            continue

# all datasets
datasets = [
    '/content/plate_train/train'
]

for dataset in datasets:
    update_classes(dataset)

print("\nFinal class mapping:")
for class_name, class_id in sorted(classes.items(), key=lambda x: x[1]):
    print(f"Class: {class_name}, ID: {class_id}")


Final class mapping:
Class: کل ناحیه پلاک, ID: 0
Class: 0, ID: 1
Class: 1, ID: 2
Class: 8, ID: 3
Class: 9, ID: 4
Class: م, ID: 5
Class: 4, ID: 6
Class: 2, ID: 7
Class: ع, ID: 8
Class: 3, ID: 9
Class: 5, ID: 10
Class: د, ID: 11
Class: 7, ID: 12
Class: 6, ID: 13
Class: و, ID: 14
Class: S, ID: 15
Class: ن, ID: 16
Class: پ, ID: 17
Class: ک, ID: 18
Class: الف, ID: 19
Class: ط, ID: 20
Class: ب, ID: 21
Class: ی, ID: 22
Class: ل, ID: 23
Class: س, ID: 24
Class: ج, ID: 25
Class: ص, ID: 26
Class: ژ (معلولین و جانبازان), ID: 27
Class: ف, ID: 28
Class: ه, ID: 29
Class: ث, ID: 30
Class: ق, ID: 31
Class: گ, ID: 32
Class: D, ID: 33
Class: تشریفات, ID: 34
Class: ش, ID: 35
Class: ت, ID: 36
Class: ز, ID: 37
Class: ه‍, ID: 38
Class: ظ, ID: 39


In [ ]:
import os
import json
import xml.etree.ElementTree as ET
import cv2

def convert_xml_to_json(folder_path):
    labels_dir = os.path.join(folder_path, "labels")
    images_dir = os.path.join(folder_path, "images")
    output_file = os.path.join(folder_path, "annotations.json")

    data = []

    for xml_file in os.listdir(labels_dir):
        if not xml_file.endswith('.xml'):
            continue

        xml_path = os.path.join(labels_dir, xml_file)
        try:
            tree = ET.parse(xml_path)
            root = tree.getroot()

            img_file = xml_file.replace('.xml', '.jpg')
            img_path = os.path.join(images_dir, img_file)
            img = cv2.imread(img_path)

            img_height, img_width = img.shape[:2]

            characters = []
            for obj in root.findall('.//object'):
                name = obj.find('name').text
                bbox = obj.find('bndbox')
                xmin = int(float(bbox.find('xmin').text))
                ymin = int(float(bbox.find('ymin').text))
                xmax = int(float(bbox.find('xmax').text))
                ymax = int(float(bbox.find('ymax').text))

                characters.append({
                    "char": name,
                    "bbox": [xmin, ymin, xmax, ymax]
                })

            characters.sort(key=lambda x: x["bbox"][0])

            plate_text = "".join([char["char"] for char in characters])

            data.append({
                "image": img_file,
                "text": plate_text,
                "characters": characters
            })

            os.remove(xml_path)

        except Exception as e:
            print(f"Error processing {xml_file}: {e}")
            continue

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

    print(f'✅ Converting process on "{folder_path}" dataset successfully done.')

for dataset in datasets:
    convert_xml_to_json(dataset)


✅ Converting process on "/content/plate_train/train" dataset successfully done.


In [ ]:
!pip install transformers datasets torch torchvision


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import json
import os
import cv2
import random
from PIL import Image
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import TrOCRProcessor, VisionEncoderDecoderModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class LicensePlateDataset(Dataset):
    def __init__(self, img_dir, label_path, processor, img_size=(128, 32)):
        self.img_dir = img_dir
        self.processor = processor
        self.img_size = img_size

        with open(label_path, "r", encoding="utf-8") as f:
            self.annotations = json.load(f)

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        data = self.annotations[idx]
        img_path = os.path.join(self.img_dir, data["image"])
        text = data["text"]

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, self.img_size)
        image = Image.fromarray(image)

        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        return pixel_values.squeeze(0), text

In [ ]:
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-small-printed")

train_dataset = LicensePlateDataset(
    img_dir="/content/plate_train/train/images",
    label_path="/content/plate_train/train/annotations.json",
    processor=processor
)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)


In [ ]:
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-small-printed")
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss(ignore_index=processor.tokenizer.pad_token_id)

from torch.cuda.amp import autocast, GradScaler
scaler = GradScaler()

config.json:   0%|          | 0.00/4.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/246M [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.deit.modeling_deit.DeiTModel'> is overwritten by shared encoder config: DeiTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 384,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "model_type": "deit",
  "num_attention_heads": 6,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.48.3"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 384,
  "d_model": 256,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 1024,
  "decode

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-13-f5554960dee7>:8: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [ ]:
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0

    for images, texts in loader:
        images = images.to(device)

        labels = processor.tokenizer(
            texts, padding="max_length", truncation=True, max_length=10, return_tensors="pt"
        ).input_ids.to(device)

        optimizer.zero_grad()

        with autocast():
            outputs = model(pixel_values=images, labels=labels)
            loss = outputs.loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

    return total_loss / len(loader)

In [ ]:
num_epochs = 3
for epoch in range(num_epochs):
    loss = train_epoch(model, train_loader, optimizer, criterion)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}")

torch.save(model.state_dict(), "trocr_finetuned.pth")
print("✅ saved")


In [ ]:
# !pip install torch torchvision torchaudio
# !pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
import os
import json
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image


In [ ]:
class LicensePlateDataset(Dataset):
    def __init__(self, img_dir, label_path, transform=None):
        self.img_dir = img_dir
        with open(label_path, "r", encoding="utf-8") as f:
            self.labels = json.load(f)
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        data = self.labels[idx]
        img_path = os.path.join(self.img_dir, data["image"])
        image = Image.open(img_path).convert("L")  # تبدیل به سیاه‌وسفید

        if self.transform:
            image = self.transform(image)

        text = data["text"]

        return image, text


In [ ]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((32, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


train_dataset = LicensePlateDataset(
    img_dir="/content/plate_train/train/images",
    label_path="/content/plate_train/train/annotations.json",
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)


In [ ]:
import torch.nn as nn
import torchvision.models as models

class CRNN(nn.Module):
    def __init__(self, num_classes):
        super(CRNN, self).__init__()
        self.cnn = models.resnet18(pretrained=True)
        self.cnn.fc = nn.Identity()
        self.rnn = nn.LSTM(512, 256, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        features = self.cnn(x)
        features = features.unsqueeze(1).repeat(1, 32, 1)
        out, _ = self.rnn(features)
        out = self.fc(out)
        return out


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_classes = 40
model = CRNN(num_classes).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CTCLoss()

def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0

    for images, texts in loader:
        images = images.to(device)

        optimizer.zero_grad()

        # ایجاد target sequence بهینه برای CTCLoss
        target_lengths = torch.tensor([len(text) for text in texts], dtype=torch.long).to(device)
        targets = torch.cat([torch.tensor([ord(c) for c in text], dtype=torch.long) for text in texts]).to(device)

        outputs = model(images)  # (batch_size, seq_length, num_classes)
        outputs = outputs.permute(1, 0, 2)  # (seq_length, batch_size, num_classes)

        input_lengths = torch.full(size=(outputs.size(1),), fill_value=outputs.size(0), dtype=torch.long).to(device)

        loss = criterion(outputs, targets, input_lengths, target_lengths)

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(loader)

num_epochs = 3
for epoch in range(num_epochs):
    loss = train_epoch(model, train_loader, optimizer, criterion)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}")

torch.save(model.state_dict(), "crnn_finetuned.pth")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 170MB/s]


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_classes = 40
model = CRNN(num_classes).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CTCLoss()

def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0

    for images, texts in loader:
        images = images.to(device)

        optimizer.zero_grad()

        # ایجاد target sequence بهینه برای CTCLoss
        target_lengths = torch.tensor([len(text) for text in texts], dtype=torch.long).to(device)
        targets = torch.cat([torch.tensor([ord(c) for c in text], dtype=torch.long) for text in texts]).to(device)

        outputs = model(images)  # (batch_size, seq_length, num_classes)
        outputs = outputs.permute(1, 0, 2)  # (seq_length, batch_size, num_classes)

        input_lengths = torch.full(size=(outputs.size(1),), fill_value=outputs.size(0), dtype=torch.long).to(device)

        loss = criterion(outputs, targets, input_lengths, target_lengths)

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(loader)

num_epochs = 3
for epoch in range(num_epochs):
    loss = train_epoch(model, train_loader, optimizer, criterion)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}")

torch.save(model.state_dict(), "crnn_finetuned.pth")


NameError: name 'torch' is not defined

In [ ]:
import os
import cv2
import numpy as np
import xml.etree.ElementTree as ET
import tensorflow as tf
import albumentations as A
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LSTM

image_folder = "/content/plate_train/train/images"
label_folder = "/content/plate_train/train/labels"

class_mapping = {
    0: "پلاک", 1: "1", 2: "9", 3: "ه", 4: "6", 5: "3", 6: "8", 7: "5", 8: "4", 9: "ج",
    10: "د", 11: "7", 12: "ط", 13: "2", 14: "ل", 15: "م", 16: "ق", 17: "و", 18: "ی",
    19: "ن", 20: "ص", 21: "س", 22: "ب", 23: "0", 24: "ت", 25: "ع", 26: "ژ", 27: "الف",
    28: "ث", 29: "پ", 30: "ظ", 31: "گ", 32: "S", 33: "ش", 34: "تشریفات", 35: "ه",
    36: "ک", 37: "D", 38: "ز", 39: "ف"
}

image_paths = []
label_paths = []
for label_file in os.listdir(label_folder):
    if label_file.endswith(".xml"):
        img_path = os.path.join(image_folder, label_file.replace(".xml", ".jpg"))
        lbl_path = os.path.join(label_folder, label_file)
        if os.path.exists(img_path) and os.path.exists(lbl_path):
            image_paths.append(img_path)
            label_paths.append(lbl_path)

def parse_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    objs = []
    for obj in root.findall("object"):
        cls_name = obj.find("name").text.strip()
        cls_id = next((k for k, v in class_mapping.items() if v == cls_name), -1)
        if cls_id == -1:
            continue
        bndbox = obj.find("bndbox")
        xmin = int(bndbox.find("xmin").text)
        ymin = int(bndbox.find("ymin").text)
        xmax = int(bndbox.find("xmax").text)
        ymax = int(bndbox.find("ymax").text)
        objs.append((cls_id, xmin, ymin, xmax, ymax))
    return objs

augmentations = A.Compose([
    A.Rotate(limit=10, p=0.7),
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
    A.GaussNoise(var_limit=50.0, p=0.4),
    A.MotionBlur(blur_limit=3, p=0.3),
    A.Affine(scale=(0.9, 1.1), translate_percent=(0.05, 0.05), rotate=(-5, 5), p=0.5),
    A.InvertImg(p=0.2),
    A.CoarseDropout(p=0.3)
])

def process_data(image_path, label_path):
    if isinstance(image_path, bytes):
        img_path = image_path.decode('utf-8')
    else:
        img_path = image_path.numpy().decode('utf-8')
    if isinstance(label_path, bytes):
        lbl_path = label_path.decode('utf-8')
    else:
        lbl_path = label_path.numpy().decode('utf-8')
    image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        return np.zeros((32,32,1), dtype=np.float32), np.zeros(len(class_mapping), dtype=np.float32)
    h, w = image.shape
    objs = parse_xml(lbl_path)
    if not objs:
        return np.zeros((32,32,1), dtype=np.float32), np.zeros(len(class_mapping), dtype=np.float32)
    for (cls_id, xmin, ymin, xmax, ymax) in objs:
        char_img = image[max(0, ymin):min(h, ymax), max(0, xmin):min(w, xmax)]
        if char_img.size == 0:
            continue
        char_img = cv2.resize(char_img, (32,32))
        aug_img = augmentations(image=char_img)["image"]
        aug_img = aug_img.astype(np.float32) / 255.0
        aug_img = np.expand_dims(aug_img, axis=-1)
        label = to_categorical(cls_id, num_classes=len(class_mapping))
        return aug_img, label
    return np.zeros((32,32,1), dtype=np.float32), np.zeros(len(class_mapping), dtype=np.float32)

def process_data_wrapper(image_path, label_path):
    img, lbl = tf.numpy_function(process_data, [image_path, label_path], [tf.float32, tf.float32])
    img.set_shape((32,32,1))
    lbl.set_shape((len(class_mapping),))
    return img, lbl

dataset = tf.data.Dataset.from_tensor_slices((image_paths, label_paths))
dataset = dataset.map(process_data_wrapper)
dataset = dataset.batch(32).shuffle(1000).prefetch(tf.data.experimental.AUTOTUNE)


model = tf.keras.Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(32,32,1), kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    Dropout(0.3),
    Conv2D(64, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    Dropout(0.3),
    Flatten(),
    Reshape((1, -1)),
    LSTM(64, return_sequences=False),
    Dropout(0.5),
    Dense(len(class_mapping), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit(dataset, epochs=3, validation_data=dataset, callbacks=[early_stopping])
model.save("/content/ocr_model_test2.h5")
print("Model saved successfully.")


<ipython-input-23-d44641abf79d>:51: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=50.0, p=0.4),


Epoch 1/3
1421/1421 [==============================] - 202s 109ms/step - loss: 1.0670 - accuracy: 0.7500 - val_loss: 0.5179 - val_accuracy: 0.8871
Epoch 2/3
1421/1421 [==============================] - 199s 110ms/step - loss: 0.6088 - accuracy: 0.8566 - val_loss: 0.4422 - val_accuracy: 0.9038
Epoch 3/3
1421/1421 [==============================] - 204s 114ms/step - loss: 0.5632 - accuracy: 0.8661 - val_loss: 0.3945 - val_accuracy: 0.9133
Model saved successfully.


/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from google.colab import files

files.download("/content/ocr_model_test2.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>